In [ ]:
import os
import numpy as np
import pandas as pd

mock_path = '/path/to/your/mock/files'  # Directory path where data files are located
column_list = ['AU24', 'AU28', 'AU45', 'Blink rate']
index_list = list(range(1, 301))

mock_output_path = '/path/to/your/output/directory/Anxiety'  # Directory path to store result files
mock_analysis_output_path = '/path/to/your/analysis/output/directory'  # Directory path to store analysis result files

T_total_result = np.zeros((144, 300, 4))  # 144: Number of Truth, 300: Number of frames, 4: Number of Anxiety metrics
F_total_result = np.zeros((72, 300, 4))  # 72: Number of Deception
f_idx = 0
t_idx = 0

for f in sorted(os.listdir(mock_path)):
    df = pd.read_csv(os.path.join(mock_path, f))
    Dataframe = df.iloc[:, 696:714]
    Anx_AUs = [13, 16, 17]  # 13 AU23, 16 AU28, 17 AU45 (Blink)
    Dataframe = Dataframe.iloc[:, Anx_AUs]
    Dataframe = Dataframe.to_numpy()
    result = np.zeros((300, 4))  # 300: Number of frames, 4: Number of Anxiety metrics

    for AU in range(3):
        for frame_len in range(2, 300):
            count = 0
            for i in range(frame_len - 1):
                if float(Dataframe[i, AU]) == 0:
                    if float(Dataframe[i, AU]) + float(Dataframe[i + 1, AU]) == 1:
                        count += 1
            result[frame_len, AU] = count

        AU45_presence_frames = sum(Dataframe[:, 2])
        Eye_blink_rate = float(AU45_presence_frames) / frame_len
        result[frame_len, 3] = Eye_blink_rate

    Anxiety_count_df = pd.DataFrame(result, columns=column_list, index=index_list)

    if not os.path.exists(mock_output_path):
        os.makedirs(mock_output_path)
    Anxiety_count_df.to_csv(os.path.join(mock_output_path, f.split('.')[0] + '.csv'))

    if f.split('.')[0].split('_')[-1] == 'T':
        T_total_result[t_idx, :, :] = result
        t_idx += 1
    elif f.split('.')[0].split('_')[-1] == 'F':
        F_total_result[f_idx, :, :] = result
        f_idx += 1



In [ ]:
import os
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind_from_stats

column_list = ['AU24', 'AU28', 'AU45', 'Blink rate']
index_list = list(range(1, 301))

# Dictionary to store necessary statistics
statistics = {}
confidence_interval = 0.95  # Confidence interval (95%)

Anxiety_stat_result_pvalue = np.zeros((300, 4))  # 300: Number of frames, 4: Number of Anxiety metrics
Anxiety_stat_result_tstat = np.zeros((300, 4))

for AU in range(4):
    for frame in range(300):
        observation_A = T_total_result[:, frame, AU]
        observation_B = F_total_result[:, frame, AU]
        observation_statistics = {}

        # T-test between Group A and Group B
        truth_mean = np.mean(T_total_result[:, frame, AU])
        truth_std = np.std(T_total_result[:, frame, AU])
        truth_sample_size = T_total_result.shape[0]

        deception_mean = np.mean(F_total_result[:, frame, AU])
        deception_std = np.std(F_total_result[:, frame, AU])
        deception_sample_size = F_total_result.shape[0]

        t_statistic, p_value = ttest_ind_from_stats(
            truth_mean, truth_std, truth_sample_size,
            deception_mean, deception_std, deception_sample_size
        )

        Anxiety_stat_result_pvalue[frame, AU] = p_value
        Anxiety_stat_result_tstat[frame, AU] = t_statistic

df1 = pd.DataFrame(Anxiety_stat_result_pvalue, columns=column_list, index=index_list)
df1.to_csv(os.path.join(mock_analysis_output_path, 'mock_Anxiety_stat_result_pvalue.csv'), index=True)
df2 = pd.DataFrame(Anxiety_stat_result_tstat, columns=column_list, index=index_list)
df2.to_csv(os.path.join(mock_analysis_output_path, 'mock_Anxiety_stat_result_tstat.csv'), index=True)
